# Cat vs NotCat dataset. 
## Conjunto de datos gato o no gato

This dataset appears in the specialized course of DeepLearning.AI about deep learning. In this case, we use Tensorflow to do the classification of if the picture is a cat or not.

Este conjunto de datos aparece en el curso especializado de DeepLearning.AI sobre apredinzaje profundo, en este caso utilizaremos tensorflow para hacer la clasificaciòn de si es un gato o no. 

In [1]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
def load_data():
    with h5py.File('train_catvnoncat.h5', "r") as train_dataset:
        train_x_orig = np.array(train_dataset["train_set_x"][:])
        train_y = np.array(train_dataset["train_set_y"][:])

    with h5py.File('test_catvnoncat.h5', "r") as test_dataset:
        test_x_orig = np.array(test_dataset["test_set_x"][:])
        test_y = np.array(test_dataset["test_set_y"][:])
        classes = np.array(test_dataset["list_classes"][:])

    train_y = train_y.reshape((1, train_y.shape[0]))
    test_y = test_y.reshape((1, test_y.shape[0]))

    return train_x_orig, train_y, test_x_orig, test_y, classes

In [3]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [4]:
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Número de imagenes de entrenamiento: " + str(m_train))
print ("Número de imagenes de testeo: " + str(m_test))
print ("Dimensión de cada imagen: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("Dimensión train_x_orig : " + str(train_x_orig.shape))
print ("Dimensión train_y: " + str(train_y.shape))
print ("Dimensión test_x_orig : " + str(test_x_orig.shape))
print ("Dimensión test_y: " + str(test_y.shape))

Número de imagenes de entrenamiento: 209
Número de imagenes de testeo: 50
Dimensión de cada imagen: (64, 64, 3)
Dimensión train_x_orig : (209, 64, 64, 3)
Dimensión train_y: (1, 209)
Dimensión test_x_orig : (50, 64, 64, 3)
Dimensión test_y: (1, 50)


### Classical gradient descent
#### Descenso del gradiente clásico

In [16]:
def modeldg():
  class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>0.99):
               print("\nSe alcanzo 99% de accuracy  cancelando entrenamiento!")
               self.model.stop_training = True
               
  train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
  train_x_orig = train_x_orig/255.0
  test_x_orig = test_x_orig/255.0
  train_r=train_y.reshape(-1,1)
  test_r=test_y.reshape(-1,1)

  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Flatten(input_shape =(64,64,3)),
                                      tf.keras.layers.Dense(20, activation= "relu"),
                                      tf.keras.layers.Dense(7, activation = "relu"),
                                      tf.keras.layers.Dense(5, activation = "relu"),
                                      tf.keras.layers.Dense(1, activation = "sigmoid")
                                      ])
  model.compile(
      loss = keras.losses.BinaryCrossentropy(from_logits = False),
      optimizer = keras.optimizers.SGD(learning_rate = 0.0075),
      metrics = ["accuracy"]
      )
  history =model.fit(train_x_orig, train_r, batch_size = 209, epochs = 2500, verbose = 1, callbacks=[callbacks], validation_data=(test_x_orig,test_r))
  return history

In [17]:
modeldg()

Epoch 1/2500
1/1 [==============================] - 1s 638ms/step - loss: 0.6998 - accuracy: 0.3397 - val_loss: 0.6839 - val_accuracy: 0.6400
Epoch 2/2500
1/1 [==============================] - 0s 50ms/step - loss: 0.6959 - accuracy: 0.4211 - val_loss: 0.6853 - val_accuracy: 0.6200
Epoch 3/2500
1/1 [==============================] - 0s 41ms/step - loss: 0.6948 - accuracy: 0.4689 - val_loss: 0.6868 - val_accuracy: 0.6400
Epoch 4/2500
1/1 [==============================] - 0s 50ms/step - loss: 0.6936 - accuracy: 0.5885 - val_loss: 0.6881 - val_accuracy: 0.6000
Epoch 5/2500
1/1 [==============================] - 0s 46ms/step - loss: 0.6929 - accuracy: 0.6124 - val_loss: 0.6893 - val_accuracy: 0.6000
Epoch 6/2500
1/1 [==============================] - 0s 45ms/step - loss: 0.6922 - accuracy: 0.6220 - val_loss: 0.6901 - val_accuracy: 0.6000
Epoch 7/2500
1/1 [==============================] - 0s 44ms/step - loss: 0.6916 - accuracy: 0.6459 - val_loss: 0.6910 - val_accuracy: 0.5600
Epoch 8/2500

### Stochastic gradiente descent
#### Descenso del gradiente estocástico

In [10]:
def modelsdg():
  class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>0.99):
               print("\nSe alcanzo 99% de accuracy  cancelando entrenamiento!")
               self.model.stop_training = True
               
  train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
  train_x_orig = train_x_orig/255.0
  test_x_orig = test_x_orig/255.0
  train_r=train_y.reshape(-1,1)
  test_r=test_y.reshape(-1,1)


  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Flatten(input_shape =(64,64,3)),
                                      tf.keras.layers.Dense(20, activation= "relu"),
                                      tf.keras.layers.Dense(7, activation = "relu"),
                                      tf.keras.layers.Dense(5, activation = "relu"),
                                      tf.keras.layers.Dense(1, activation = "sigmoid")
                                      ])
  model.compile(
      loss = keras.losses.BinaryCrossentropy(from_logits = False),
      optimizer = keras.optimizers.SGD(learning_rate = 0.0075),
      metrics = ["accuracy"]
      )
  history =model.fit(train_x_orig, train_r, batch_size = 1, epochs = 2500, verbose = 1, callbacks=[callbacks], validation_data=(test_x_orig,test_r))
  return history


In [11]:
modelsdg()

Epoch 1/2500
209/209 [==============================] - 1s 4ms/step - loss: 0.6901 - accuracy: 0.6316 - val_loss: 0.7249 - val_accuracy: 0.3400
Epoch 2/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.6545 - accuracy: 0.6507 - val_loss: 0.7081 - val_accuracy: 0.4000
Epoch 3/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.6472 - accuracy: 0.6555 - val_loss: 0.7190 - val_accuracy: 0.3400
Epoch 4/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.6318 - accuracy: 0.6651 - val_loss: 0.7003 - val_accuracy: 0.3800
Epoch 5/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.6250 - accuracy: 0.6507 - val_loss: 0.7004 - val_accuracy: 0.4400
Epoch 6/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.5879 - accuracy: 0.6507 - val_loss: 0.7097 - val_accuracy: 0.3400
Epoch 7/2500
209/209 [==============================] - 1s 3ms/step - loss: 0.5794 - accuracy: 0.6603 - val_loss: 0.7500 - val_accuracy:

### Mini-batch gradient descendt
### Descenso del gradiente por mini lotes

In [18]:
def modelmb():
  class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>0.99):
               print("\nSe alcanzo 99% de accuracy  cancelando entrenamiento!")
               self.model.stop_training = True
               
  train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
  train_x_orig = train_x_orig/255.0
  test_x_orig = test_x_orig/255.0
  train_r=train_y.reshape(-1,1)
  test_r=test_y.reshape(-1,1)


  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Flatten(input_shape =(64,64,3)),
                                      tf.keras.layers.Dense(20, activation= "relu"),
                                      tf.keras.layers.Dense(7, activation = "relu"),
                                      tf.keras.layers.Dense(5, activation = "relu"),
                                      tf.keras.layers.Dense(1, activation = "sigmoid")
                                      ])
  model.compile(
      loss = keras.losses.BinaryCrossentropy(from_logits = False),
      optimizer = keras.optimizers.SGD(learning_rate = 0.0075),
      metrics = ["accuracy"]
      )
  history =model.fit(train_x_orig, train_r, batch_size = 100, epochs = 2500, verbose = 1, callbacks=[callbacks],validation_data=(test_x_orig,test_r))
  return history

In [19]:
modelmb()

Epoch 1/2500
3/3 [==============================] - 1s 105ms/step - loss: 0.6834 - accuracy: 0.5837 - val_loss: 0.7349 - val_accuracy: 0.4000
Epoch 2/2500
3/3 [==============================] - 0s 23ms/step - loss: 0.6633 - accuracy: 0.6507 - val_loss: 0.7611 - val_accuracy: 0.3600
Epoch 3/2500
3/3 [==============================] - 0s 28ms/step - loss: 0.6534 - accuracy: 0.6507 - val_loss: 0.7337 - val_accuracy: 0.3400
Epoch 4/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6589 - accuracy: 0.6507 - val_loss: 0.7493 - val_accuracy: 0.3600
Epoch 5/2500
3/3 [==============================] - 0s 23ms/step - loss: 0.6525 - accuracy: 0.6555 - val_loss: 0.8174 - val_accuracy: 0.3600
Epoch 6/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6382 - accuracy: 0.6555 - val_loss: 0.8120 - val_accuracy: 0.3600
Epoch 7/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6376 - accuracy: 0.6555 - val_loss: 0.9123 - val_accuracy: 0.3400
Epoch 8/2500

### Adam optimizer
#### Optimizador Adam

In [12]:
def modeladam():
  class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>0.99):
               print("\nSe alcanzo 99% de accuracy  cancelando entrenamiento!")
               self.model.stop_training = True
               
  train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
  train_x_orig = train_x_orig/255.0
  test_x_orig = test_x_orig/255.0
  train_r=train_y.reshape(-1,1)
  test_r=test_y.reshape(-1,1)


  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Flatten(input_shape =(64,64,3)),
                                      tf.keras.layers.Dense(20, activation= "relu"),
                                      tf.keras.layers.Dense(7, activation = "relu"),
                                      tf.keras.layers.Dense(5, activation = "relu"),
                                      tf.keras.layers.Dense(1, activation = "sigmoid")
                                      ])
  model.compile(
      loss = keras.losses.BinaryCrossentropy(from_logits = False),
      optimizer = keras.optimizers.Adam(learning_rate = 0.0075),
      metrics = ["accuracy"]
      )
  history =model.fit(train_x_orig, train_r, batch_size = 100, epochs = 2500, verbose = 1, callbacks=[callbacks],validation_data=(test_x_orig,test_r))
  return history

In [13]:
modeladam()

Epoch 1/2500
3/3 [==============================] - 1s 160ms/step - loss: 2.5365 - accuracy: 0.6603 - val_loss: 1.3365 - val_accuracy: 0.3400
Epoch 2/2500
3/3 [==============================] - 0s 24ms/step - loss: 1.0009 - accuracy: 0.6555 - val_loss: 0.7231 - val_accuracy: 0.3400
Epoch 3/2500
3/3 [==============================] - 0s 23ms/step - loss: 0.6890 - accuracy: 0.6555 - val_loss: 0.6999 - val_accuracy: 0.3400
Epoch 4/2500
3/3 [==============================] - 0s 29ms/step - loss: 0.6874 - accuracy: 0.6555 - val_loss: 0.6941 - val_accuracy: 0.3400
Epoch 5/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6921 - accuracy: 0.6555 - val_loss: 0.6954 - val_accuracy: 0.3400
Epoch 6/2500
3/3 [==============================] - 0s 50ms/step - loss: 0.6907 - accuracy: 0.6555 - val_loss: 0.6974 - val_accuracy: 0.3400
Epoch 7/2500
3/3 [==============================] - 0s 34ms/step - loss: 0.6888 - accuracy: 0.6555 - val_loss: 0.6999 - val_accuracy: 0.3400
Epoch 8/2500

### RMS optimizer
#### Optimizador RMS

In [14]:
def modelRMS():
  class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get("accuracy")>0.99):
               print("\nSe alcanzo 99% de accuracy  cancelando entrenamiento!")
               self.model.stop_training = True
               
  train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
  train_x_orig = train_x_orig/255.0
  test_x_orig = test_x_orig/255.0
  train_r=train_y.reshape(-1,1)
  test_r=test_y.reshape(-1,1)


  callbacks = myCallback()
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.Flatten(input_shape =(64,64,3)),
                                      tf.keras.layers.Dense(20, activation= "relu"),
                                      tf.keras.layers.Dense(7, activation = "relu"),
                                      tf.keras.layers.Dense(5, activation = "relu"),
                                      tf.keras.layers.Dense(1, activation = "sigmoid")
                                      ])
  model.compile(
      loss = keras.losses.BinaryCrossentropy(from_logits = False),
      optimizer = keras.optimizers.RMSprop(learning_rate = 0.0075),
      metrics = ["accuracy"]
      )
  history =model.fit(train_x_orig, train_r, batch_size = 100, epochs = 2500, verbose = 1, callbacks=[callbacks],validation_data=(test_x_orig,test_r))
  return history

In [15]:
modelRMS()

Epoch 1/2500
3/3 [==============================] - 1s 119ms/step - loss: 7.0074 - accuracy: 0.3397 - val_loss: 0.6576 - val_accuracy: 0.7200
Epoch 2/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6946 - accuracy: 0.5455 - val_loss: 0.7012 - val_accuracy: 0.3400
Epoch 3/2500
3/3 [==============================] - 0s 23ms/step - loss: 0.6855 - accuracy: 0.6555 - val_loss: 0.7041 - val_accuracy: 0.3400
Epoch 4/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6832 - accuracy: 0.6555 - val_loss: 0.7064 - val_accuracy: 0.3400
Epoch 5/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6814 - accuracy: 0.6555 - val_loss: 0.7073 - val_accuracy: 0.3400
Epoch 6/2500
3/3 [==============================] - 0s 25ms/step - loss: 0.6807 - accuracy: 0.6555 - val_loss: 0.7113 - val_accuracy: 0.3400
Epoch 7/2500
3/3 [==============================] - 0s 23ms/step - loss: 0.6779 - accuracy: 0.6555 - val_loss: 0.7128 - val_accuracy: 0.3400
Epoch 8/2500